In [1]:
# Imports
from dask_jobqueue import SLURMCluster
from distributed import Client
from run_cocycles_ot import run

In [2]:
# Cluster creation
cluster = SLURMCluster(
    n_workers=0,
    memory="16GB",
    processes=1,
    cores=1,
    scheduler_options={
        "dashboard_address": ":10095",
        "allowed_failures": 10
    },
    job_cpu=1,
    walltime="24:0:0",
    #job_extra_directives = ["-p medium,fast,cpu"],
)
cluster.adapt(minimum=0, maximum=100)
client = Client(cluster)

In [3]:
# Submitting jobs
n = 500
ntrial = 20
corrs = [0.1,0.3,0.5,0.7,0.9]
additive = False
multivariate = False
futures = []
learn_rate = 1e-2
for corr in corrs:
    for seed in range(ntrial):
        f0 = client.submit(run,n = n, seed = seed, corr = corr, additive = additive, multivariate_noise = multivariate, learn_rate = learn_rate)
        futures += [f0]

In [4]:
futures

[<Future: pending, key: run-0da6654cc555153c4be25a8a5177c6bf>,
 <Future: pending, key: run-e88f8ebb82e79ac4dc2a36c6c3b1dc63>,
 <Future: pending, key: run-73d196c64e9c0947bb75971b9f6ad0e1>,
 <Future: pending, key: run-dacd290590f72799d8ca790761df4579>,
 <Future: pending, key: run-8bb7fb97a9258fe1291e706752041eba>,
 <Future: pending, key: run-4dccd0b3a08cfad6fc08cf5278734eba>,
 <Future: pending, key: run-6a3bbee8daf81c49c04744487c1343ab>,
 <Future: pending, key: run-c11847a4c8e44b0002cf655494ebaa8d>,
 <Future: pending, key: run-682ae65dc6ea119aacaf3ec0f37689d9>,
 <Future: pending, key: run-cb59a7ed69cdbacd4739afa0b2df49dd>,
 <Future: pending, key: run-1ada0a848dfe19c340dd7e5fdc147e15>,
 <Future: pending, key: run-8559994231932cbd88f412b32e610d11>,
 <Future: pending, key: run-60cfe1953d48ed3ae5417f8faa7fc16d>,
 <Future: pending, key: run-f90991ade5a6ffd8f2ccf5705883e868>,
 <Future: pending, key: run-cf310ac4090a87dbf89e97b176a519bd>,
 <Future: pending, key: run-517dc2e9ab1401764879cb92ca4

In [5]:
# Gettaing results
results = client.gather(futures)

In [6]:
# Closing client
client.close()
cluster.close()

2025-08-15 01:19:50,989 - distributed.deploy.adaptive_core - INFO - Adaptive stop


In [7]:
import torch
torch.save(f = "nfcocycle_cv_results_trials={0}_n={1}_additive={2}_lr={3}_new_.pt".format(ntrial,n,additive, learn_rate), obj = results)